In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout , Flatten , Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [24]:
from google.colab import drive

In [25]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [26]:
!unzip -uq "/content/drive/MyDrive/facemask/archive.zip" -d "/content/drive/MyDrive/facemask/archive"

In [27]:
import os
main_dir = '/content/drive/MyDrive/facemask/New Masks Dataset'
train_dir = os.path.join(main_dir,'Train')
test_dir = os.path.join(main_dir,'Test')
valid_dir = os.path.join(main_dir,'Validation')

train_mask_dir = os.path.join(train_dir,'Mask')
train_nomask_dir = os.path.join(train_dir,'Non mask')

In [28]:
print(train_dir)

/content/drive/MyDrive/facemask/New Masks Dataset/Train


In [29]:
train_mask_names = os.listdir(train_mask_dir)
print(train_mask_names[:10])
train_nomask_dir = os.listdir('/content/drive/MyDrive/facemask/New Masks Dataset/Train/Non Mask')
print(train_nomask_dir[:10])

['0003.jpg', '0006.jpg', '0018.jpg', '0019.jpg', '0022.jpg', '0027.jpg', '0029.jpg', '0037.jpg', '0040.jpg', '0042.jpg']
['0.jpg', '1.jpg', '10.jpg', '100.jpg', '101.jpg', '102.jpg', '104.jpg', '105.jpg', '106.jpg', '107.jpg']


In [30]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range = 0.2,
                                   rotation_range = 40,
                                   horizontal_flip= True
                                   )
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150,150),
                                                    batch_size = 32,
                                                    class_mode= 'binary'
                                                    )


test_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(150,150),
                                                    batch_size = 32,
                                                    class_mode='binary'
                                                    )


valid_generator = validation_datagen.flow_from_directory(valid_dir,
                                                    target_size=(150,150),
                                                    batch_size = 32,
                                                    class_mode='binary'
                                                    )

Found 600 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Found 306 images belonging to 2 classes.


In [31]:
train_generator.class_indices

{'Mask': 0, 'Non Mask': 1}

In [32]:
train_generator.image_shape

(150, 150, 3)

In [33]:
model = Sequential()
model.add(Conv2D(32,(3,3),padding='SAME',activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(64,(3,3),padding='SAME',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 75, 75, 32)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 75, 75, 32)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 37, 37, 64)       

In [34]:
model.compile(Adam(lr=0.001),loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [35]:
history = model.fit(train_generator,
                    epochs = 30,
                    validation_data=valid_generator)
                    

Epoch 1/30
19/19 [==============================] - 41s 2s/step - loss: 2.2355 - accuracy: 0.5717 - val_loss: 0.6925 - val_accuracy: 0.5686
Epoch 2/30
19/19 [==============================] - 38s 2s/step - loss: 0.5932 - accuracy: 0.7100 - val_loss: 0.5502 - val_accuracy: 0.8235
Epoch 3/30
19/19 [==============================] - 39s 2s/step - loss: 0.3998 - accuracy: 0.8300 - val_loss: 0.4374 - val_accuracy: 0.8856
Epoch 4/30
19/19 [==============================] - 38s 2s/step - loss: 0.3104 - accuracy: 0.8900 - val_loss: 0.3600 - val_accuracy: 0.9118
Epoch 5/30
19/19 [==============================] - 39s 2s/step - loss: 0.2987 - accuracy: 0.8883 - val_loss: 0.4346 - val_accuracy: 0.8431
Epoch 6/30
19/19 [==============================] - 38s 2s/step - loss: 0.2630 - accuracy: 0.9050 - val_loss: 0.3244 - val_accuracy: 0.9314
Epoch 7/30
19/19 [==============================] - 38s 2s/step - loss: 0.2783 - accuracy: 0.9000 - val_loss: 0.3533 - val_accuracy: 0.9150
Epoch 8/30
19/19 [==

In [36]:
test_loss , test_acc = model.evaluate(test_generator)
print('test acc :{} test loss :{}'.format(test_acc,test_loss))

4/4 [==============================] - 4s 769ms/step - loss: 0.1715 - accuracy: 0.9400
test acc :0.9399999976158142 test loss :0.1715400069952011


In [42]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()
#print(uploaded)
for f in uploaded.keys():
  img_path='/content/' +f
  img = image.load_img(img_path , target_size=(150,150))
  images = image.img_to_array(img)
  images = np.expand_dims(images,axis=0)
  prediction = model.predict(images)
  if prediction==0:
    print(f,'mask is there')
  else:
    print(f,'no mask is there')

Saving man.jpg to man (2).jpg
man.jpg no mask is there


In [44]:
model.save('newsaved_model1.h5')